In [12]:
%pip install -r requirements.txt





ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL
Note: you may need to restart the kernel to use updated packages.


In [43]:
import torch
from transformers import (
    AutoModel,
    AutoImageProcessor,
    AutoFeatureExtractor,
    ViTFeatureExtractor,
    ViTModel,
)
import torch.nn.functional as F


# from torchvision.transforms import Resize, ToTensor
from torchvision.transforms.functional import to_pil_image
from PIL import Image
from torchvision import transforms
import requests
from io import BytesIO

imageBirdURL = "https://static.inaturalist.org/photos/405494470/original.jpeg"
imageFlowerURL = (
    "https://inaturalist-open-data.s3.amazonaws.com/photos/405552903/original.jpeg"
)

responseBird = requests.get(imageBirdURL)
responseFlower = requests.get(imageFlowerURL)

imageBird = Image.open(BytesIO(responseBird.content))
imageFlower = Image.open(BytesIO(responseFlower.content))

# resize each image
imageBird.thumbnail((100, 100))
imageBird.save("resized_image_bird.jpg")

imageFlower.thumbnail((100, 100))
imageFlower.save("resized_image_flower.jpg")
# vision transformer ViT
featureExtract = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
model = ViTModel.from_pretrained("google/vit-base-patch16-224")

inputTensorsBird = featureExtract(images=imageBird, return_tensors="pt")
inputTensorsFlower = featureExtract(images=imageFlower, return_tensors="pt")

with torch.no_grad():
    outputBird = model(**inputTensorsBird)
    outputFlower = model(**inputTensorsFlower)

last_hidden_state_bird = outputBird.last_hidden_state
last_hidden_state_flower = outputFlower.last_hidden_state

# reshape to W x H x C tensor
reshaped_features_bird = last_hidden_state_bird[0, 1:, :].reshape(14, 14, 768)
reshaped_features_flower = last_hidden_state_flower[0, 1:, :].reshape(14, 14, 768)


l2NormBird = F.normalize(reshaped_features_bird, p=2, dim=-1)
l2NormFlower = F.normalize(reshaped_features_flower, p=2, dim=-1)

avgFeatureBird = l2NormBird.mean(dim=1)
avgFeatureFlower = l2NormFlower.mean(dim=1)

sumFeatureBird = avgFeatureBird.sum(dim=-1)
sumFeatureFlower = avgFeatureFlower.sum(dim=-1)

print(l2NormBird)


## EXTRACT TENSORS ##
# resizeBird = featureExtract(
#     images=imageBird, size=(100, 100), return_tensors="pt"
# ).pixel_values
# resizeFlower = featureExtract(
#     images=imageFlower, size=(100, 100), return_tensors="pt"
# ).pixel_values

# resized_image_Bird = Image.fromarray(
#     (resizeBird[0].permute(1, 2, 0) * 255).byte().numpy()
# )
# resized_image_Bird.save("resized_image_bird.jpg")
# resized_image_Flower = Image.fromarray(
#     (resizeFlower[0].permute(1, 2, 0) * 255).byte().numpy()
# )
# resized_image_Flower.save("resized_image_flower.jpg")

/Users/anuragkej/extractVisualFeatures/.conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/anuragkej/extractVisualFeatures/.conda/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<built-in method sum of Tensor object at 0x14d3e1130>
